In [1]:
import pandas as pd

df = pd.read_csv('/Users/JooAnLee/mid_project/src/datasets/수도권_아파트_청약_뉴스_크롤링_청약기간내.csv')
df.head(5)

,공고번호,apartment,title,content,url
0,2025000043,부천 JY 포에시아,'안동용상하늘채리버스카이' 등 1205가구 분양예정[분양캘린더],견본주택 '용현우방아이유쉘센트럴마린' 개관[서울=뉴시스][서울=뉴시스]정진형 기자 ...,https://n.news.naver.com/mnews/article/003/001...
1,2025000043,부천 JY 포에시아,다음 주 전국 분양물량 1205가구에 그쳐…서울은 '無',다음 주 분양 물량이 전주의 절반에도 미치지 못하는 1200여가구에 그칠 것으로 보...,https://n.news.naver.com/mnews/article/277/000...
2,2025000043,부천 JY 포에시아,분양 성수기인데…내주 전국서 1205가구 공급 그쳐,부동산R114 주간 부동산 분양 캘린더봄철 분양 성수기에도 분양 시장은 침체기가 이...,https://n.news.naver.com/mnews/article/015/000...
3,2025000020,e편한세상 제물포역 파크메종(조합원 취소분),"3월 첫째주, 전국 2334가구 청약…지방 물량 집중",전국 6곳 청약 접수서울·인천 수도권 2곳3월 첫째 주 전국 6곳에서 총 2334가...,https://n.news.naver.com/mnews/article/277/000...
4,2025000020,e편한세상 제물포역 파크메종(조합원 취소분),[주간분양] 수도권 청약 물량 희귀…전국 2334가구 중 35가구,전국 6곳 중 4곳이 지방…2299가구 공급28일 리얼투데이에 따르면 3월 첫째 주...,https://n.news.naver.com/mnews/article/119/000...


In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from konlpy.tag import Okt
import re

In [3]:
# 코퍼스 생성 (content 컬럼 데이터를 리스트로 변환)
corpus = df['content'].tolist()
print(corpus[:3])

["견본주택 '용현우방아이유쉘센트럴마린' 개관[서울=뉴시스][서울=뉴시스]정진형 기자 = 부동산R114는 3월 둘째 주 전국 3개 단지 총 1205가구가 분양을 시작한다고 8일 밝혔다. 모두 일반분양 물량이다.청약 접수는 경기 부천시 심곡동 '부천JY포에시아', 경북 안동시 용상동 '안동용상하늘채리버스카이', 울산 울주군 삼남읍 '울산에피트스타시티' 등에서 진행된다. 모델하우스는 인천 미추홀구 용현동 ‘용현우방아이유쉘센트럴마린’ 1곳이 개관한다.코오롱글로벌은 경북 안동시 용상동 일원에 '안동용상하늘채리버스카이'를 분양한다. 지하 2층~지상 최고 26층, 9개동, 전용면적 84㎡, 총 548가구 규모다. 단지 앞 경동로를 통한 안동 시내·외 이동이 편리하며 용상교리신국도를 비롯한 용상중로, 안동 외곽순환도로 교차로 등이 개통되면 교통 여건이 개선될 전망이다. 선어대 생태공원, 낙천공원, 낙동강 등 자연환경이 풍부하며 안동다목적체육관, 안동문화관광단지가 자리해 생활 편의성이 좋다.SM스틸·SM경남기업은 인천 미추홀구 용현동 일원에서 새한아파트 주택재건축을 통해 주상복합 아파트 '용현우방아이유쉘센트럴마린'을 분양할 예정이다. 지하 3층~지상 최고 41층, 2개동, 전용면적 59~76㎡, 총 252가구 중 아파트 196가구(오피스텔 76실 별도)를 일반분양한다. 수인분당선 숭의역을 이용할 수 있고 제2경인고속도로, 수도권제2순환고속도로 등 광역 교통망도 잘 갖춰져 있다. 신선초, 신흥중, 신흥여중 등 학교가 가깝고 인하대병원, 옹진군청, 대형마트 등 기반 시설도 인접한다．", '다음 주 분양 물량이 전주의 절반에도 미치지 못하는 1200여가구에 그칠 것으로 보인다.8일 부동산R114에 따르면 오는 10일부터 16일까지 전국에서 3개 단지, 1205가구가 분양된다. 이는 전주 2753가구와 비교하면 56% 감소한 수치다.서울은 분양 물량이 전무하다. 지난달 래미안 원페를라 공급을 마지막으로 5주째 일반 분양 물량이 없다.공급 예정인 3개 단지는 경기 부천시 심곡동 \'

In [4]:
# 형태소 분석기 설정
okt = Okt()

# 불용어 정리
with open('stopwords-ko.txt', 'r') as f:
  list_file = f.readlines() 
stopwords_default = [word[:-1] for word in list_file ]
stopwords_default
stopwords = stopwords_default # 기본

# 1. 텍스트 정제 함수 (특수문자, 숫자 제거)
def clean_text(text):
    text = re.sub(r'\[.*?\]|\(.*?\)', '', text) # (), [] 괄호 안 내용 제거
    text = re.sub(r'[^가-힣\s]', '', text)  # 한글과 공백 제외 문자 제거
    text = re.sub(r'\s+', ' ', text).strip()  # 연속 공백 제거
    return text

# 2. 형태소 분석을 통한 명사 추출 함수
def extract_nouns(text):
    nouns = okt.nouns(text)  # 형태소만 추출
    nouns = [word for word in nouns if word not in stopwords and len(word) > 1]  # 불용어 제거 및 한 글자 단어 제외
    return ' '.join(nouns)

# 3. 전체 데이터 전처리
corpus = df['content'].tolist()
cleaned_corpus = [extract_nouns(clean_text(text)) for text in corpus]  # 정제 + 명사 추출

In [5]:
cleaned_corpus[:5]

['견본주택 우방 아이유 센트럴 마린 개관 정진 기자 부동산 둘째 전국 가구 분양 시작 일반 분양 물량 청약 접수 경기 부천시 심곡동 부천 시아 경북 안동시 용상동 안동 용상 하늘 리버 스카이 울산 울주군 삼남 울산 피트 스타 시티 진행 모델하우스 인천 미추홀 용현동 우방 아이유 센트럴 마린 개관 코오롱 글로벌 경북 안동시 용상동 일원 안동 용상 하늘 리버 스카이 분양 지하 지상 최고 전용 면적 가구 규모 경동로 통한 안동 내외 이동 용상 교리 국도 비롯 용상 안동 외곽순환 도로 교차로 개통 교통 여건 개선 전망 선어 생태 공원 낙천 공원 낙동강 자연환경 동다 목적 체육관 안동 문화 관광 자리 생활 편의 스틸 경남 기업 인천 미추홀 용현동 일원 아파트 주택 재건축 통해 주상 복합 아파트 우방 아이유 센트럴 마린 분양 예정 지하 지상 최고 전용 면적 가구 아파트 가구 일반 수인 분당선 숭의역 이용 경인 고속도로 수도권 순환 고속도로 광역 교통 신선 신흥 신흥 여중 학교 인하대병원 옹진군 대형 마트 기반 시설 인접',
 '분양 물량 전주 절반 가구 부동산 전국 가구 분양 전주 가구 비교 감소 수치 서울 분양 물량 지난달 래미안 원페를 공급 마지막 일반 분양 물량 공급 예정 경기 부천시 심곡동 부천 시아 경북 안동시 용상동 안동 용상 하늘 리버 스카이 울산 울주군 삼남 울산 피트 스타 시티 부동산 계획 분양 물량 일부 건설 미분 우려 분양 일정 조정 청약 온기 기대 확산',
 '부동산 주간 부동산 분양 캘린더 봄철 분양 수기 분양 시장 침체 내주 공급물량 가구 전망 부동산 내주 전국 가구 가구 절반 수준 지난해 한주 최대 가구 공급 기도 일반 분양 수기 서울 제외 지역 청약 경쟁률 정치 불확실 성과 경기 침체 지속 전반 청약 심리 위축 공급 물량 감소 분석 서울 분양 물량 서울 지난달 첫째 래미안 원페를 공급 마지막 일반 분양 물량 상황 지속 공급 경기 부천시 원미구 심곡동 부천 시아 경북 안동시 용상동 안동 용상 하늘 리버 스카이 울산 울주군 삼남 울산 피트 스타

In [6]:
# CountVectorizer로 전처리한 기사 텍스트를 빈도수 기반 단어 추출
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
    max_df=0.9,  # 전체 문서의 80% 이상 등장한 단어 제외
    min_df=10,    # Minimum Document Frequency, 최소 문서 빈도 = min_df=5면 최소 5개 문서에서 등장해야 포함됨
    max_features=1000,
    ngram_range=(1, 3),  # unigram + bigram + trigram 사용
    token_pattern=r'\b\w+\b'  # 숫자 포함 단어도 추출 가능
)

dtm = vectorizer.fit_transform(cleaned_corpus)  # 전처리된 데이터로 DTM 생성
dtm.shape

# 추출된 단어 리스트 출력
words = vectorizer.get_feature_names_out()
print("✅ 추출된 키워드 목록:", words[:1000])

✅ 추출된 키워드 목록: ['가가' '가격' '가격 경쟁력' '가구' '가구 가구' '가구 가구 가구' '가구 공급' '가구 구성' '가구 규모'
 '가구 규모 조성' '가구 모집' '가구 모집 신청' '가구 모집 접수' '가구 모집 청약' '가구 모집 평균' '가구 분양'
 '가구 일반' '가구 일반 분양' '가구 전용' '가구 조성' '가구 청약' '가구 청약 접수' '가까이' '가능성' '가도'
 '가든' '가량' '가로' '가운데' '가입' '가입 기간' '가장' '가점' '가족' '가치' '감소' '감정' '감정 청약'
 '강남' '강남구' '강동' '강동구' '강북구' '강원' '강화' '개관' '개발' '개발 사업' '개발 호재' '개방' '개선'
 '개월' '개통' '개통 예정' '거래' '거리' '거실' '거주' '거주 의무' '건립' '건설' '건설 경기' '건설 경기도'
 '건의' '건의 청약' '견본주택' '견본주택 개관' '견본주택 오픈' '결과' '결과 가구' '결과 가구 모집' '경기'
 '경기도' '경남' '경우' '경의' '경인' '경인 고속도로' '경쟁' '경쟁력' '경쟁률' '경쟁률 기록' '경쟁률 전용'
 '경제' '경제 무단' '경제 무단 배포' '계속' '계약' '계약 일간' '계약 일간 진행' '계약 진행' '계약금' '계양'
 '계획' '고가' '고객' '고덕' '고려' '고속도로' '고양' '고양시' '곤지암' '골프' '골프 연습장' '공간' '공개'
 '공고' '공급' '공급 가구' '공급 가구 모집' '공급 물량' '공급 순위' '공급 시작' '공급 시작 순위' '공급 예정'
 '공급 청약' '공사' '공원' '과천' '관계자' '관련' '관심' '광명' '광역' '광역급행' '광역급행 철도' '광주'
 '광주시' '교육' '교육 환경' '교통' '구가' '구간' '구독' '구성' '구역' '구역 재개발' '구조' '국제'
 '국토교통부' '규모' '규모 가구' '규모 조성' '규제' '규제 완화' '규제 지역

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pandas as pd
import pprint

# 최적의 하이퍼파라미터로 LDA 모델 학습 (토픽 수 7개)
lda = LatentDirichletAllocation(
    n_components=7, 
    doc_topic_prior=0.2, 
    topic_word_prior=0.01,
    max_iter=10,
    learning_method='batch',
    random_state=42
)
lda.fit(dtm)

# 토픽 이름: 실제 토픽 수에 맞춰 "토픽 1" ~ "토픽 7" 사용
topic_names = [f'토픽 {i}' for i in range(1, 8)]
terms = vectorizer.get_feature_names_out()
topic_keywords = {}

# 토픽별 상위 10개 키워드 추출
for i, topic in enumerate(lda.components_):
    top_keywords = [terms[j] for j in topic.argsort()[-10:][::-1]]
    topic_keywords[topic_names[i]] = top_keywords

# 각 기사별 토픽 분포 (토픽 점수) 계산
doc_topic = lda.transform(dtm)

results = []
for i, topic_dist in enumerate(doc_topic):
    top_topic = topic_dist.argmax()
    result = {
        '기사 번호': i + 1,
        '주요 토픽': topic_names[top_topic],
    }
    # 각 토픽에 대한 점수를 소수점 4자리로 반올림하여 결과에 추가
    result.update({topic_names[j]: round(topic_dist[j], 4) for j in range(len(topic_names))})
    results.append(result)

df_results = pd.DataFrame(results)

# 결과 출력
print("\n[토픽별 주요 키워드]")
pprint.pprint(topic_keywords)

print("\n[기사별 주요 토픽 점수]")
df_results.head()


[토픽별 주요 키워드]
{'토픽 1': ['공급', '전용', '억원', '아파트', '대출', '주택', '가구', '억만원', '적용', '서울'],
 '토픽 2': ['경쟁률', '가구', '순위', '모집', '기록', '전용', '가구 모집', '평균', '경쟁률 기록', '접수'],
 '토픽 3': ['세대', '순위', '공급', '아파트', '진행', '순위 청약', '당첨', '계약', '타입', '발표'],
 '토픽 4': ['당첨', '가점', '주택', '지역', '통장', '서울', '최고', '공급', '전용', '평균'],
 '토픽 5': ['아파트', '서울', '시장', '경쟁률', '가구', '순위', '부동산', '지역', '수도권', '미달'],
 '토픽 6': ['가구', '예정', '시설', '가구 가구', '순위', '공원', '조성', '위치', '지역', '고속도로'],
 '토픽 7': ['가구', '전용', '접수', '지상', '경기', '지하', '면적', '전용 면적', '지하 지상', '청약 접수']}

[기사별 주요 토픽 점수]


,기사 번호,주요 토픽,토픽 1,토픽 2,토픽 3,토픽 4,토픽 5,토픽 6,토픽 7
0,1,토픽 7,0.0016,0.0016,0.0016,0.0016,0.0016,0.2367,0.7555
1,2,토픽 7,0.0040,0.0039,0.0039,0.0040,0.4600,0.0040,0.5202
2,3,토픽 5,0.0023,0.0023,0.0023,0.0023,0.6225,0.0023,0.3660
3,4,토픽 7,0.0016,0.0016,0.0016,0.0016,0.1261,0.0016,0.8660
4,5,토픽 7,0.0009,0.0009,0.0010,0.0009,0.0010,0.0010,0.9943


In [ ]:
# 토픽별 상위 20개 키워드 추출
for i, topic in enumerate(lda.components_):
    top_keywords = [terms[j] for j in topic.argsort()[-40:][::-1]]
    topic_keywords[topic_names[i]] = top_keywords

# 결과 출력
print("\n[토픽별 주요 키워드]")
pprint.pprint(topic_keywords)


[토픽별 주요 키워드]
{'토픽 1': ['공급',
          '전용',
          '억원',
          '아파트',
          '대출',
          '주택',
          '가구',
          '억만원',
          '적용',
          '서울',
          '시세',
          '가가',
          '분양 가가',
          '가격',
          '규제',
          '파주',
          '특별',
          '특별 공급',
          '물량',
          '거래',
          '상한',
          '인근',
          '거주',
          '중도',
          '분양 상한',
          '기준',
          '운정신도시',
          '시장',
          '최근',
          '부동산',
          '지역',
          '주변',
          '경우',
          '운정',
          '책정',
          '수도권',
          '택지',
          '일반',
          '의무',
          '대비'],
 '토픽 2': ['경쟁률',
          '가구',
          '순위',
          '모집',
          '기록',
          '전용',
          '가구 모집',
          '평균',
          '경쟁률 기록',
          '접수',
          '순위 청약',
          '최고',
          '평균 경쟁률',
          '진행',
          '공급',
          '마감',
          '타입',
          '최고 경쟁률',
          '면적',
      

In [ ]:
# 찾고자 하는 키워드
keyword = '가격'

# 해당 키워드를 포함하는 토픽 찾기
found_topic = None
for topic, keywords in topic_keywords.items():
    if keyword in keywords:
        found_topic = topic
        break

if found_topic:
    print(f"'{keyword}' 키워드는 '{found_topic}'에 속합니다.")
else:
    print(f"'{keyword}' 키워드는 찾을 수 없습니다.")

'가격' 키워드는 '토픽 1'에 속합니다.


In [ ]:
df_combined = pd.concat([df, df_results], axis=1)
df_combined.head()

,공고번호,apartment,title,content,url,기사 번호,주요 토픽,토픽 1,토픽 2,토픽 3,토픽 4,토픽 5,토픽 6,토픽 7
0,2025000043,부천 JY 포에시아,'안동용상하늘채리버스카이' 등 1205가구 분양예정[분양캘린더],견본주택 '용현우방아이유쉘센트럴마린' 개관[서울=뉴시스][서울=뉴시스]정진형 기자 ...,https://n.news.naver.com/mnews/article/003/001...,1,토픽 7,0.0016,0.0016,0.0016,0.0016,0.0016,0.2367,0.7555
1,2025000043,부천 JY 포에시아,다음 주 전국 분양물량 1205가구에 그쳐…서울은 '無',다음 주 분양 물량이 전주의 절반에도 미치지 못하는 1200여가구에 그칠 것으로 보...,https://n.news.naver.com/mnews/article/277/000...,2,토픽 7,0.0040,0.0039,0.0039,0.0040,0.4600,0.0040,0.5202
2,2025000043,부천 JY 포에시아,분양 성수기인데…내주 전국서 1205가구 공급 그쳐,부동산R114 주간 부동산 분양 캘린더봄철 분양 성수기에도 분양 시장은 침체기가 이...,https://n.news.naver.com/mnews/article/015/000...,3,토픽 5,0.0023,0.0023,0.0023,0.0023,0.6225,0.0023,0.3660
3,2025000020,e편한세상 제물포역 파크메종(조합원 취소분),"3월 첫째주, 전국 2334가구 청약…지방 물량 집중",전국 6곳 청약 접수서울·인천 수도권 2곳3월 첫째 주 전국 6곳에서 총 2334가...,https://n.news.naver.com/mnews/article/277/000...,4,토픽 7,0.0016,0.0016,0.0016,0.0016,0.1261,0.0016,0.8660
4,2025000020,e편한세상 제물포역 파크메종(조합원 취소분),[주간분양] 수도권 청약 물량 희귀…전국 2334가구 중 35가구,전국 6곳 중 4곳이 지방…2299가구 공급28일 리얼투데이에 따르면 3월 첫째 주...,https://n.news.naver.com/mnews/article/119/000...,5,토픽 7,0.0009,0.0009,0.0010,0.0009,0.0010,0.0010,0.9943


In [ ]:
# 그룹별 평균 계산
grouped_df_combined = df_combined.groupby(['공고번호', 'apartment']).agg({
    '기사 번호': 'min',  # 기사 번호 중 최소값 사용
    '토픽 1': 'mean',
    '토픽 2': 'mean',
    '토픽 3': 'mean',
    '토픽 4': 'mean',
    '토픽 5': 'mean',
    '토픽 6': 'mean',
    '토픽 7': 'mean'
}).reset_index()

# 주요 토픽 설정
grouped_df_combined['주요 토픽'] = grouped_df_combined[['토픽 1', '토픽 2', '토픽 3', '토픽 4', '토픽 5', '토픽 6', '토픽 7']].idxmax(axis=1)

grouped_df_combined.tail()

,공고번호,apartment,기사 번호,토픽 1,토픽 2,토픽 3,토픽 4,토픽 5,토픽 6,토픽 7,주요 토픽
683,2024000730,래미안 센트리폴(2BL),19,0.014967,0.043433,0.262400,0.146900,0.258300,0.001267,0.272767,토픽 7
684,2025000001,래미안 원페를라,7,0.146933,0.090867,0.001267,0.719333,0.002967,0.001267,0.037367,토픽 4
685,2025000004,양주 용암 영무 예다음 더퍼스트,13,0.198900,0.016133,0.242733,0.048633,0.000900,0.440100,0.052500,토픽 6
686,2025000020,e편한세상 제물포역 파크메종(조합원 취소분),4,0.020867,0.001167,0.001200,0.001167,0.050333,0.001200,0.924133,토픽 7
687,2025000043,부천 JY 포에시아,1,0.002633,0.002600,0.002600,0.002633,0.361367,0.081000,0.547233,토픽 7


In [ ]:
# Save the grouped_df_combined DataFrame to a CSV file
grouped_df_combined.to_csv('토픽모델링_최종.csv', index=False)

In [ ]:
grouped_df_combined.rename(columns={'apartment': '주택명'}, inplace=True)
grouped_df_combined.head()

,공고번호,주택명,기사 번호,토픽 1,토픽 2,토픽 3,토픽 4,토픽 5,토픽 6,토픽 7,주요 토픽
0,2020000277,인천검단신도시3차 노블랜드 리버파크,1939,0.001467,0.223500,0.001467,0.624567,0.145900,0.001467,0.001467,토픽 4
1,2020000295,영통자이,1931,0.089800,0.023567,0.099000,0.337967,0.154033,0.277033,0.018633,토픽 4
2,2020000298,시흥장현 영무예다음,1951,0.062200,0.157967,0.000900,0.517567,0.168600,0.007433,0.085333,토픽 4
3,2020000306,부평역 한라비발디 트레비앙,1928,0.001133,0.452833,0.046333,0.090200,0.407300,0.001100,0.001100,토픽 2
4,2020000311,안산 푸르지오 브리파크,1945,0.001533,0.266233,0.060100,0.512633,0.090367,0.020367,0.048700,토픽 4


In [ ]:
train_data = pd.read_csv('/Users/JooAnLee/mid_project/src/storage/train_data/train-250317-01.csv', encoding='euc-kr')
train_data.head()

,공고번호,주택명,공급지역코드,공급지역명,공급위치우편번호,공급위치,공급규모,모집공고일,청약접수시작일,청약접수종료일,...,위도,경도,행정동코드,법정동코드,시도,시군구,읍면동1,읍면동2,전용면적당 공급금액(최고가기준),전용면적당 시세차익
0,2025000043,부천 JY 포에시아,410,경기,14636,경기도 부천시 원미구 신흥로 20,55,2025-02-28,2025-03-10,2025-03-11,...,37.485377,126.776922,4.119252e+09,4.119210e+09,경기,부천시 원미구,심곡2동,심곡동,8.767442e+06,NaN
1,2025000043,부천 JY 포에시아,410,경기,14636,경기도 부천시 원미구 신흥로 20,55,2025-02-28,2025-03-10,2025-03-11,...,37.485377,126.776922,4.119252e+09,4.119210e+09,경기,부천시 원미구,심곡2동,심곡동,8.767442e+06,NaN
2,2025000043,부천 JY 포에시아,410,경기,14636,경기도 부천시 원미구 신흥로 20,55,2025-02-28,2025-03-10,2025-03-11,...,37.485377,126.776922,4.119252e+09,4.119210e+09,경기,부천시 원미구,심곡2동,심곡동,8.767442e+06,NaN
3,2025000043,부천 JY 포에시아,410,경기,14636,경기도 부천시 원미구 신흥로 20,55,2025-02-28,2025-03-10,2025-03-11,...,37.485377,126.776922,4.119252e+09,4.119210e+09,경기,부천시 원미구,심곡2동,심곡동,8.767442e+06,NaN
4,2025000043,부천 JY 포에시아,410,경기,14636,경기도 부천시 원미구 신흥로 20,55,2025-02-28,2025-03-10,2025-03-11,...,37.485377,126.776922,4.119252e+09,4.119210e+09,경기,부천시 원미구,심곡2동,심곡동,8.767442e+06,NaN


In [ ]:
# 병합
merged_df_topic = pd.merge(train_data, grouped_df_combined, on=['공고번호', '주택명'], how='left')
merged_df_topic.head()

,공고번호,주택명,공급지역코드,공급지역명,공급위치우편번호,공급위치,공급규모,모집공고일,청약접수시작일,청약접수종료일,...,전용면적당 시세차익,기사 번호,토픽 1,토픽 2,토픽 3,토픽 4,토픽 5,토픽 6,토픽 7,주요 토픽
0,2025000043,부천 JY 포에시아,410,경기,14636,경기도 부천시 원미구 신흥로 20,55,2025-02-28,2025-03-10,2025-03-11,...,NaN,1,0.002633,0.0026,0.0026,0.002633,0.361367,0.081,0.547233,토픽 7
1,2025000043,부천 JY 포에시아,410,경기,14636,경기도 부천시 원미구 신흥로 20,55,2025-02-28,2025-03-10,2025-03-11,...,NaN,1,0.002633,0.0026,0.0026,0.002633,0.361367,0.081,0.547233,토픽 7
2,2025000043,부천 JY 포에시아,410,경기,14636,경기도 부천시 원미구 신흥로 20,55,2025-02-28,2025-03-10,2025-03-11,...,NaN,1,0.002633,0.0026,0.0026,0.002633,0.361367,0.081,0.547233,토픽 7
3,2025000043,부천 JY 포에시아,410,경기,14636,경기도 부천시 원미구 신흥로 20,55,2025-02-28,2025-03-10,2025-03-11,...,NaN,1,0.002633,0.0026,0.0026,0.002633,0.361367,0.081,0.547233,토픽 7
4,2025000043,부천 JY 포에시아,410,경기,14636,경기도 부천시 원미구 신흥로 20,55,2025-02-28,2025-03-10,2025-03-11,...,NaN,1,0.002633,0.0026,0.0026,0.002633,0.361367,0.081,0.547233,토픽 7


In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17250 entries, 0 to 17249
Data columns (total 41 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   공고번호               17250 non-null  int64  
 1   주택명                17250 non-null  object 
 2   공급지역코드             17250 non-null  int64  
 3   공급지역명              17250 non-null  object 
 4   공급위치우편번호           17250 non-null  int64  
 5   공급위치               17250 non-null  object 
 6   공급규모               17250 non-null  int64  
 7   모집공고일              17250 non-null  object 
 8   청약접수시작일            17250 non-null  object 
 9   청약접수종료일            17250 non-null  object 
 10  당첨자발표일             17250 non-null  object 
 11  투기과열지구             17250 non-null  object 
 12  조정대상지역             17250 non-null  object 
 13  분양가상한제             17250 non-null  object 
 14  정비사업               17250 non-null  object 
 15  공공주택지구             17250 non-null  object 
 16  대규모택지개발지구          172

In [ ]:
merged_df_topic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17250 entries, 0 to 17249
Data columns (total 50 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   공고번호               17250 non-null  int64  
 1   주택명                17250 non-null  object 
 2   공급지역코드             17250 non-null  int64  
 3   공급지역명              17250 non-null  object 
 4   공급위치우편번호           17250 non-null  int64  
 5   공급위치               17250 non-null  object 
 6   공급규모               17250 non-null  int64  
 7   모집공고일              17250 non-null  object 
 8   청약접수시작일            17250 non-null  object 
 9   청약접수종료일            17250 non-null  object 
 10  당첨자발표일             17250 non-null  object 
 11  투기과열지구             17250 non-null  object 
 12  조정대상지역             17250 non-null  object 
 13  분양가상한제             17250 non-null  object 
 14  정비사업               17250 non-null  object 
 15  공공주택지구             17250 non-null  object 
 16  대규모택지개발지구          172

In [ ]:
# CSV 파일로 저장
merged_df_topic.to_csv('train250317-01_topic_merged.csv', index=False)

## 모델 저장

In [ ]:
import joblib

# 모델 저장 및 로드

version = '0.0.1'
# 모델 저장
joblib.dump(vectorizer, f"./storage/topic_modeling/countvectorizer_model_{version}.pkl")
joblib.dump(lda, f"./storage/topic_modeling/lda_model_{version}.pkl")

# 모델 로드
loaded_vectorizer = joblib.load(f"./storage/topic_modeling/countvectorizer_model_{version}.pkl")
loaded_lda = joblib.load(f"./storage/topic_modeling/lda_model_{version}.pkl")

In [ ]:
# 모델 파일 존재 여부 확인
model_directory = './storage/topic_modeling'
model_files = os.listdir(model_directory)
print(model_files)

['countvectorizer_model_0.0.1.pkl', 'lda_model_0.0.1.pkl']


### 테스트

In [ ]:
# 복사한 텍스트
text = """국내 최초 공동주택 수직증축 리모델링 단지인 '잠실 더샵 루벤'이 준공을 마치고 입주에 나섰다.

17일 포스코이앤씨에 따르면 이 단지는 서울시 송파구 송파동에 위치한 1992년에 준공된 성지 아파트를 수직증축으로 리모델링해 완성했따. 2020년에 사업계획이 승인된 후 2022년 1월 착공하여 총 38개월의 공사 끝에 2025년 3월 새롭게 탄생했다. 기존 2개 동 지하 2층~지상 15층, 298가구에서 지하 3층~지상 18층, 327가구로 탈바꿈했다.

기존 가구의 전용면적은 리모델링을 통해 66㎡에서 83㎡, 84㎡에서 106㎡로 확장됐다. 팬트리와 드레스룸을 추가하고 희망 가구에 대면형 주방을 적용해서 더욱 다양한 생활공간을 제공했다. 또한 기존 세대에 우물 천장 설계를 적용해 천장고를 최대한 확보했다.

잠실 더샵 루벤 입주 예정자는 "펜트리, 드레스룸 등이 있는 신축과 같이 바뀌어 다양한 공간활용이 가능해졌다" 며 "특히 가구에서 지하주차장까지 엘리베이터가 직접 연결되고 부족했던 주차 공간이 2배 가까이 늘어나면서 출퇴근 때마다 느끼던 스트레스에서 해방될 것 같다"고 말했다.

기술적 측면에서 보면 잠실 더샵 루벤은 국내에서 처음으로 3개층을 수직증축한 단지다. '수직증축'은 기존 아파트의 층을 위로 연장하는 리모델링 방식이다. 기존 층수가 15층 미만은 최대 2개층, 15층 이상이면 최대 3개층까지 증축할 수 있다. 수직 증축은 두 차례의 안전성 검토를 거치는 등 구조 안전에 더욱 엄격한 기준을 적용하기 때문에 높은 기술력을 요구한다. 포스코이앤씨는 3개층 수직증축에 따른 추가하중을 단단한 지반에 분담하도록 난이도 높은 기초 보강 공사를 실시했다. 기존 구조물 하부를 굴착해야 하는 엘리베이터 설치 구간에는 보강 말뚝으로 지지하는 '뜬구조공법'을 적용했고, 지지력이 부족한 구간은 단단한 지반까지 기초를 내리는 '내림기초 보강 공법', 추가 콘크리트 타설로 기초를 두껍게 하는 '기초증타 보강 공법'으로 기초를 견고히 했다.

또 신구(新舊) 구조물을 더 견고히 연결하기 위해 두 구조물의 철근을 용접하고 케미컬 앵커로 고정하는 '신구 연결 복합공법'을 적용했다. 3D 스캐닝으로 기존 건축물을 정밀 측정해 BIM(Building Information Modeling)으로 시뮬레이션하고 실시간 모니터링 계측시스템을 도입하는 등 스마트 건설 기술도 활용함으로써 최적의 철거 방안을 검토하고 정밀한 시공으로 안전성을 확보했다.

한편 이 단지는 수직증축을 통해 29가구가 일반 분양 물량으로 나왔다. 최초 공급 당시 3.3㎡당 분양가는 6500만원으로, 전용 106㎡ 타입별 분양가는 25억7440만~26억4700만원에 달했다. 당시 대출이 나오지 않는 고가 아파트 기준인 '15억원'을 훌쩍 넘는 금액으로, 당첨자는 대출을 끼지 않고 자체적으로 자금을 조달해야 했다. 분양 물량이 30가구 미만이면 분양가 상한제 등 각종 규제를 피할 수 있어 조합에서 분양가를 높게 책정한 것이다.

청약에서 7310명이 몰려 평균 경쟁률이 252.1대 1이 나왔지만 결국 대거 미계약 물량이 나왔다. 이후 지난해 전용 106㎡ 분양가를 기존보다 낮춰 19억3000여만원에 공급했다. 23가구 임의공급을 했는데 630명이 신청해 평균 경쟁률 27.3 대 1을 기록했다. 현재 29가구 모두 분양이 완료됐다."""

# 텍스트를 데이터프레임으로 변환
test_data = {'content': [text]}
test_df = pd.DataFrame(test_data)

print("테스트 데이터프레임:")
print(test_df)

# 저장된 모델 불러오기
vectorizer_path = '../storage/topic_modeling/countvectorizer_model_0.0.1.pkl'
lda_model_path = '../storage/topic_modeling/lda_model_0.0.1.pkl'

try:
    loaded_vectorizer = joblib.load(vectorizer_path)
    loaded_lda = joblib.load(lda_model_path)
except FileNotFoundError:
    print("모델 파일이 존재하지 않습니다. 경로를 확인하세요.")
else:
    # 모델이 정상적으로 로드되면 테스트 진행
    print("모델 로드 성공")

    # 테스트 데이터 벡터화
    test_vector = loaded_vectorizer.transform(test_df['content'])

    # 토픽 분포 계산
    topic_distribution = loaded_lda.transform(test_vector)

    # 결과 출력
    print("\n토픽 분포:")
    print(topic_distribution)

테스트 데이터프레임:
                                             content
0  국내 최초 공동주택 수직증축 리모델링 단지인 '잠실 더샵 루벤'이 준공을 마치고 입...
모델 로드 성공

토픽 분포:
[[0.46768059 0.08677755 0.29950302 0.00453128 0.00453981 0.13242459
  0.00454316]]


## 다시 병합

In [ ]:
train_data2 = pd.read_csv('/Users/JooAnLee/mid_project/src/storage/train_data/train-250317-03.csv', encoding='euc-kr')
train_data2.head()

,공고번호,주택명,공급지역코드,공급규모,투기과열지구,조정대상지역,분양가상한제,정비사업,공공주택지구,대규모택지개발지구,...,순위,거주지역,접수건수,경쟁률,최저당첨가점,전용면적,미달여부,공급금액(최고가 기준),법정동코드,전용면적당 시세차익
0,2025000043,부천 JY 포에시아,410,55,N,N,N,N,N,N,...,1순위,해당지역,0,0.0,0.0,43.7207,Y,377000000.0,4.119210e+09,NaN
1,2025000043,부천 JY 포에시아,410,55,N,N,N,N,N,N,...,1순위,기타지역,15,3.0,0.0,43.7207,N,377000000.0,4.119210e+09,NaN
2,2025000043,부천 JY 포에시아,410,55,N,N,N,N,N,N,...,2순위,해당지역,3,0.0,0.0,43.7207,Y,377000000.0,4.119210e+09,NaN
3,2025000043,부천 JY 포에시아,410,55,N,N,N,N,N,N,...,2순위,기타지역,10,0.0,0.0,43.7207,Y,377000000.0,4.119210e+09,NaN
4,2025000043,부천 JY 포에시아,410,55,N,N,N,N,N,N,...,1순위,해당지역,1,0.2,0.0,43.7207,Y,377000000.0,4.119210e+09,NaN


In [ ]:
# 병합
merged_df_topic2 = pd.merge(train_data2, grouped_df_combined, on=['공고번호', '주택명'], how='left')
merged_df_topic2.head()

,공고번호,주택명,공급지역코드,공급규모,투기과열지구,조정대상지역,분양가상한제,정비사업,공공주택지구,대규모택지개발지구,...,전용면적당 시세차익,기사 번호,토픽 1,토픽 2,토픽 3,토픽 4,토픽 5,토픽 6,토픽 7,주요 토픽
0,2025000043,부천 JY 포에시아,410,55,N,N,N,N,N,N,...,NaN,1,0.002633,0.0026,0.0026,0.002633,0.361367,0.081,0.547233,토픽 7
1,2025000043,부천 JY 포에시아,410,55,N,N,N,N,N,N,...,NaN,1,0.002633,0.0026,0.0026,0.002633,0.361367,0.081,0.547233,토픽 7
2,2025000043,부천 JY 포에시아,410,55,N,N,N,N,N,N,...,NaN,1,0.002633,0.0026,0.0026,0.002633,0.361367,0.081,0.547233,토픽 7
3,2025000043,부천 JY 포에시아,410,55,N,N,N,N,N,N,...,NaN,1,0.002633,0.0026,0.0026,0.002633,0.361367,0.081,0.547233,토픽 7
4,2025000043,부천 JY 포에시아,410,55,N,N,N,N,N,N,...,NaN,1,0.002633,0.0026,0.0026,0.002633,0.361367,0.081,0.547233,토픽 7


In [ ]:
merged_df_topic2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17250 entries, 0 to 17249
Data columns (total 31 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   공고번호          17250 non-null  int64  
 1   주택명           17250 non-null  object 
 2   공급지역코드        17250 non-null  int64  
 3   공급규모          17250 non-null  int64  
 4   투기과열지구        17250 non-null  object 
 5   조정대상지역        17250 non-null  object 
 6   분양가상한제        17250 non-null  object 
 7   정비사업          17250 non-null  object 
 8   공공주택지구        17250 non-null  object 
 9   대규모택지개발지구     17250 non-null  object 
 10  수도권내민영공공주택지구  17250 non-null  object 
 11  공급세대수         17250 non-null  int64  
 12  순위            17250 non-null  object 
 13  거주지역          17250 non-null  object 
 14  접수건수          17250 non-null  int64  
 15  경쟁률           17250 non-null  float64
 16  최저당첨가점        17250 non-null  float64
 17  전용면적          17250 non-null  float64
 18  미달여부          17250 non-nu

In [ ]:
# CSV 파일로 저장
merged_df_topic2.to_csv('train250317-03_topic_merged.csv', index=False)

## coherence score 계산

In [8]:
pip install gensim


Note: you may need to restart the kernel to use updated packages.


In [10]:
from gensim import corpora
from gensim.models import CoherenceModel

# 1. gensim에 맞는 DTM 형식으로 변환 (위에서 만든 `cleaned_corpus` 이용)
# 먼저, gensim의 DTM을 만들기 위해서는 List of List 형태로 텍스트 데이터를 변환해야 합니다.
# 예: [["단어1", "단어2", "단어3"], ["단어4", "단어5", "단어6"], ...]

# DTM을 gensim에 맞게 변환
corpus_gensim = [text.split() for text in cleaned_corpus]

# 2. 사전 만들기 (단어 → ID 매핑)
dictionary = corpora.Dictionary(corpus_gensim)

# 3. 텍스트 데이터를 Bow 형식으로 변환
bow_corpus = [dictionary.doc2bow(text) for text in corpus_gensim]

# 4. LDA 모델 학습 (gensim의 LdaModel을 사용)
from gensim.models import LdaModel

# gensim LDA 모델 학습
lda_gensim = LdaModel(corpus=bow_corpus, id2word=dictionary, num_topics=7, random_state=42)

# 5. Coherence Score 계산 (c_v Coherence Score 방식 사용)
coherence_model_lda = CoherenceModel(model=lda_gensim, corpus=bow_corpus, dictionary=dictionary, texts=corpus_gensim, coherence='c_v')
coherence_score = coherence_model_lda.get_coherence()

print(f"✅ LDA 모델의 Coherence Score: {coherence_score:.4f}")


✅ LDA 모델의 Coherence Score: 0.4498
